In [20]:
using PauliPropagation
using Random
using Optim
using Plots
Random.seed!(43)
using ReverseDiff: GradientTape, gradient!, compile, gradient
using LinearAlgebra
using StatsBase 
using GLM
using DataFrames
using CSV
using Format

# CDR for quantum dynamics
- TFIM Hamiltonian (constants not site-dependent) evolving with TDSE
- discretize the evolution steps and use first-order Trotter decomposition 

### Idea:
- DONE: Run "exact" evolution for small trotterized circuit
- DONE: Create near-Clifford circuits:replace some gates in Trotterized gate by close Cliffords - no VQA here so this is much simpler, find closest Clifford to the RZZ and to the RX (only have 2 angles in total), replace a portion randomly (no MCMC), keep the parameter of N = non-Cliffords const. st. system remains cl. scalable.
- create MWE of CDR within quantum dynamics
- which gates ones we replace has influence on the accuracy of the expectation value (principle of causal light cone as seen in vnCDR paper Piotr, is this automatically respected within Heisenberg picture? (backprop observable)), so we can aim to replace only gates that contribute to an expectation value (works only for single-qubit / local observables)
- another idea is to try the perturbation approach (all non-Cliffords) 



In [2]:
struct trotter_ansatz
    target_circuit::Vector{Gate}
    topology::Vector{Tuple{Int64, Int64}}
    nqubits::Integer
    steps::Integer
    time::Integer
    J::Float64
    h::Float64
    sigma_J::Float64
    sigma_h::Float64
    sigma_J_indices::Vector{Int64}
    sigma_h_indices::Vector{Int64}
end

In [3]:
function trotter_setup(nqubits::Integer, steps::Integer, time::Float64, J::Float64, h::Float64;topology = nothing)
    if isnothing(topology)
        topology = bricklayertopology(nqubits)
    end
    target_circuit = tfitrottercircuit(nqubits,steps,topology=topology) #start with RZZ layer
   
    sigma_J = -2*T*J/steps
    sigma_h = 2*T*h/steps 

    sigma_J_indices = getparameterindices(target_circuit, PauliRotation, [:Z,:Z]) 
    sigma_h_indices = getparameterindices(target_circuit, PauliRotation, [:X])
    
    return trotter_ansatz(target_circuit, topology, nqubits, steps, time, J, h,sigma_J, sigma_h,sigma_J_indices, sigma_h_indices)
end

trotter_setup (generic function with 1 method)

In [4]:
function constrain_params(ansatz)
    nparams = countparameters(ansatz.target_circuit)
    thetas = zeros(nparams)
    thetas[ansatz.sigma_h_indices] .= ansatz.sigma_h
    thetas[ansatz.sigma_J_indices] .= ansatz.sigma_J
    return thetas
end

constrain_params (generic function with 1 method)

In [5]:
function obs_magnetization(ansatz)
    magnetization = PauliSum(ansatz.nqubits)
    for i in 1:nq
        add!(magnetization,:Z,i)
    end
    magnetization = magnetization/nq
    return magnetization
end

obs_magnetization (generic function with 1 method)

In [6]:
function exact_trotter_time_evolution(ansatz)
    thetas = constrain_params(ansatz)
    obs = obs_magnetization(ansatz)
    circuit = copy(ansatz.target_circuit)
    psum =propagate(circuit,obs, thetas)
    return overlapwithzero(psum)
end

exact_trotter_time_evolution (generic function with 1 method)

In [7]:
function training_set_generation_brut(ansatz::trotter_ansatz, angle_definition::Float64; num_samples::Int = 10, non_cliffs::Int = 30)
    nparams = countparameters(ansatz.target_circuit)
    cliffs = nparams - non_cliffs
    ratio = length(ansatz.sigma_J_indices)/(length(ansatz.sigma_h_indices)+length(ansatz.sigma_J_indices))
    num_h = Int(round((1-ratio)*cliffs))
    num_J = Int(round(ratio*cliffs))
    training_thetas_list = Vector{Vector{Float64}}()
    thetas = constrain_params(ansatz)
    for _ in 1:num_samples
        training_thetas = deepcopy(thetas)
        shuffled_sigma_h_indices =  Random.shuffle!(ansatz.sigma_h_indices)
        shuffled_sigma_J_indices = Random.shuffle!(ansatz.sigma_J_indices)
        selected_indices_h = shuffled_sigma_h_indices[1:num_h]
        selected_indices_J = shuffled_sigma_J_indices[1:num_J];   
        k_h =round(ansatz.sigma_h/(angle_definition))
        k_J =round(ansatz.sigma_J/(angle_definition))

        for i in selected_indices_h
            training_thetas[i] = k_h*angle_definition
        end
        for i in selected_indices_J
            training_thetas[i] = k_J*angle_definition
        end
        push!(training_thetas_list, training_thetas)
    end
    return training_thetas_list
end

training_set_generation_brut (generic function with 1 method)

In [8]:
function training_exact_time_evolution_brut(ansatz, training_thetas, min_abs_coeff)
    exact_expvals = Vector{Float64}()
    for thetas in training_thetas
        obs = obs_magnetization(ansatz)
        circuit = copy(ansatz.target_circuit)
        psum =propagate(circuit,obs, thetas; min_abs_coeff=min_abs_coeff)
        push!(exact_expvals, overlapwithzero(psum))
    end
    return exact_expvals
end

training_exact_time_evolution_brut (generic function with 1 method)

In [9]:
function noisy_time_evolution_brut(ansatz, exact_training_thetas, min_abs_coeff=0; depol_strength = 0.05, dephase_strength = 0.05)
    noisy_expvals = Vector{Float64}()

    #to be replaced with a decent noise model
    depol_noise_layer = [DepolarizingNoise(qind, depol_strength ) for qind in 1:ansatz.nqubits];
    dephase_noise_layer = [DephasingNoise(qind, dephase_strength) for qind in 1:ansatz.nqubits];
    
    noisy_circuit = deepcopy(ansatz.target_circuit)
    obs = obs_magnetization(ansatz)
    append!(noisy_circuit,depol_noise_layer)
    append!(noisy_circuit,dephase_noise_layer)

    for i in 1:length(exact_training_thetas)
        thetas = exact_training_thetas[i]
        psum = propagate(noisy_circuit,obs,thetas; min_abs_coeff=min_abs_coeff)
        push!(noisy_expvals, overlapwithzero(psum))
    end

    return noisy_expvals
end

noisy_time_evolution_brut (generic function with 2 methods)

### Exact evolution of a small trotterized circuit (see CPDR p.7)

In [10]:
# nq = 32
# steps = 10
# T = 1.0
# J = 5.0 #J > 0 in ferromagnetic phase, J < 0 in antiferromagnetic phase
# h = 1.0 #abs(h) < abs(J) in ordered phase
# trotter = trotter_setup(nq, steps, T, J, h);

# min_abs_coeff = 1e-6;
# min_abs_coeff_noisy = min_abs_coeff;

In [11]:
# exact_expval_target = exact_trotter_time_evolution(trotter) #should be close to one as we stay in FM phase

In [12]:
# noisy_expval_target = noisy_time_evolution_brut(trotter, [constrain_params(trotter)], min_abs_coeff_noisy)

### Training set generation

In [13]:
# num_samples = 10
# angle_definition = pi/8
# list = training_set_generation_brut(trotter, angle_definition; num_samples = 10, non_cliffs = 30);

### Exact time evolution for the training set

In [14]:

# exact_expvals = training_exact_time_evolution_brut(trotter,list,min_abs_coeff);

### Noisy expectation values
- 1 layer of depolarizing and dephasing noise
- for more realistic noise model, add after each gate

In [15]:
# noisy_expvals = noisy_time_evolution_brut(trotter,list,min_abs_coeff_noisy);

### CDR

In [16]:
function cdr(noisy_exp_values::Vector{Float64}, exact_exp_values::Vector{Float64}, noisy_target_exp_value::Float64, exact_target_exp_value::Float64; verbose=false)
    training_data = DataFrame(x=noisy_exp_values,y=exact_exp_values)
    ols = lm(@formula(y ~ x), training_data)
    function cdr_em(x)
        return  coef(ols)[1] + coef(ols)[2] * x
    end
    rel_error_after = abs(exact_target_exp_value - cdr_em(noisy_target_exp_value)) / abs(exact_target_exp_value)
    rel_error_before = abs(exact_target_exp_value - noisy_target_exp_value) / abs(exact_target_exp_value)
    if verbose
        println(training_data)
        println("Noisy target expectation value: ", noisy_target_exp_value)
        println("Relative error before CDR: ", rel_error_before)
        println("CDR-EM target expectation value: ", cdr_em(noisy_target_exp_value))
        println("Relative error after CDR: ", rel_error_after)
    end
    return cdr_em(noisy_target_exp_value), rel_error_after, rel_error_before
end 

cdr (generic function with 1 method)

In [ ]:
# corr_energy, rel_error_after, rel_error_before = cdr(noisy_expvals, exact_expvals, noisy_expval_target[1], exact_expval_target; verbose=true)  

UndefVarError: UndefVarError: `noisy_expval_target` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [45]:
nq = 4
steps = 10
angle_definition = pi/8
T = 1.0
J = 5.0 #J > 0 in ferromagnetic phase, J < 0 in antiferromagnetic phase
h = 1.0 #abs(h) < abs(J) in ordered phase
trotter = trotter_setup(nq, steps, T, J, h);

min_abs_coeff = 1e-6;
min_abs_coeff_noisy = min_abs_coeff;

printfmt("ready to ruuuuuummmble\n")
time1 = time()
exact_expval_target = exact_trotter_time_evolution(trotter) #should be close to one as we stay in FM phase
timetmp1 = time()
printfmt("exact_expval_target done in {:.2f} s\n", timetmp1-time1)

noisy_expval_target = noisy_time_evolution_brut(trotter, [constrain_params(trotter)], min_abs_coeff_noisy)
timetmp2 = time()
printfmt("noisy_expval_target done in {:.2f} s\n", timetmp2-timetmp1)
timetmp1 = timetmp2

num_samples = 10
angle_definition = pi/8
list = training_set_generation_brut(trotter, angle_definition; num_samples = 10, non_cliffs = 30);
timetmp2 = time()
printfmt("training_set_generation_brut done in {:.2f} s\n", timetmp2-timetmp1)
timetmp1 = timetmp2

exact_expvals = training_exact_time_evolution_brut(trotter,list,min_abs_coeff);
timetmp2 = time()
printfmt("training_exact_time_evolution_brut done in {:.2f} s\n", timetmp2-timetmp1)
timetmp1 = timetmp2

noisy_expvals = noisy_time_evolution_brut(trotter,list,min_abs_coeff_noisy);
timetmp2 = time()
printfmt("noisy_time_evolution_brut done in {:.2f} s\n", timetmp2-timetmp1)
timetmp1 = timetmp2

corr_energy, rel_error_after, rel_error_before = cdr(noisy_expvals, exact_expvals, noisy_expval_target[1], exact_expval_target; verbose=true)  
timetmp2 = time()
printfmt("cdr done in {:.2f} s\n", timetmp2-timetmp1)


printfmt("total run done in {:.2f} s\n", timetmp2-time1)

# open file in append mode 
log = open("trotter_brut.log", "a")  

str = format("{:>5n} {:>5n} {:>6.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e} {:>10.2e}\n", nq, steps, T, J, h, angle_definition, min_abs_coeff, min_abs_coeff_noisy, exact_expval_target, noisy_expval_target[1], rel_error_before, rel_error_after, timetmp2-time1);
# writing to a file using write() method  
write(log, str)  
    
# We need to close the file in order to write the content from the disk to file  
close(log)


ready to ruuuuuummmble
exact_expval_target done in 0.00 s
noisy_expval_target done in 0.00 s
training_set_generation_brut done in 0.00 s
training_exact_time_evolution_brut done in 0.00 s
noisy_time_evolution_brut done in 0.00 s
10×2 DataFrame
 Row │ x         y        
     │ Float64   Float64  
─────┼────────────────────
   1 │ 0.796926  0.83887
   2 │ 0.834355  0.878268
   3 │ 0.859226  0.904449
   4 │ 0.814003  0.856846
   5 │ 0.736201  0.774948
   6 │ 0.814137  0.856986
   7 │ 0.829869  0.873547
   8 │ 0.887828  0.934556
   9 │ 0.792039  0.833725
  10 │ 0.81328   0.856085
Noisy target expectation value: 0.9000170340165776
Relative error before CDR: 0.04999999581718834
CDR-EM target expectation value: 0.9473863515963972
Relative error after CDR: 4.402959295487908e-9
cdr done in 0.00 s
total run done in 0.01 s
